###Initiation

In [ ]:
import kagglehub
arshiahemmat_dataset_ldp_path = kagglehub.dataset_download('arshiahemmat/dataset-ldp')

print('Data source import complete.')


In [ ]:
!pip install -q langchain langchain-community langchain-openai langchain-google-genai docx2txt json5 faiss-cpu

In [ ]:
!gdown --id "12dE8fwuOjS8NcZ1AdZpG2vZlZXf_2Xv8"
!unzip Data.zip

In [ ]:
from kaggle_secrets import UserSecretsClient
secret_label = "avalai_api"
API_KEY = UserSecretsClient().get_secret(secret_label)
GOOGLE_API_KEY = UserSecretsClient().get_secret(google_api_key)

###NormalRAG

In [ ]:
import os
import json
from langchain_community.document_loaders import TextLoader, Docx2txtLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.schema import HumanMessage, SystemMessage



class NormalRAG:
    def __init__(self, llm_name="openai", embedding_model="openai", api_key=None, model_name=None, base_url=None):
        self.llm_name = llm_name
        self.embedding_model = embedding_model
        self.api_key = api_key
        self.model_name = model_name
        self.base_url = base_url or "https://api.avalai.ir/v1"  # Default base URL for Gemini

        # Initialize LLM
        if self.llm_name == "openai":
            if not self.api_key:
                raise ValueError("OpenAI API key is required for llm_name='openai'")
            self.llm = ChatOpenAI(api_key=self.api_key, model_name=self.model_name, base_url=self.base_url)
        elif self.llm_name == "gemini":
            if not self.api_key:
                raise ValueError("Gemini API key is required for llm_name='gemini'")
            self.llm = ChatGoogleGenerativeAI(google_api_key=self.api_key, model=self.model_name)
        else:
            raise ValueError("Invalid llm_name. Choose 'openai' or 'gemini'.")

        # Initialize embeddings based on the independent `embedding_model`
        if self.embedding_model == "openai":
            self.embeddings = OpenAIEmbeddings(api_key=self.api_key, base_url=self.base_url)
        elif self.embedding_model == "gemini":
            self.embeddings = GoogleGenerativeAIEmbeddings(google_api_key=self.api_key)
        else:
            raise ValueError("Invalid embedding_model. Choose 'openai' or 'gemini'.")

        # Initialize vector store and text splitter
        self.vectorstore = None
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.vector_store_path = "RAG_Database"

    def json_name(self):
        json_name = f"{rag_model_name}_{self.llm_name}_{self.embeddings.__class__.__name__}.jsonl"
        return json_name

    def load_documents(self, dataset_folder):
        if not os.path.exists(dataset_folder):
            print(f"Dataset folder '{dataset_folder}' not found.")
            return

        documents = []

        for filename in os.listdir(dataset_folder):
            folder_path = os.path.join(dataset_folder, filename)
            if os.path.isdir(folder_path):
                for i in range(1, 11):
                    filepath = os.path.join(folder_path, f'{i}.docx')

                    if os.path.exists(filepath):
                        loader = Docx2txtLoader(filepath)
                        documents.extend(loader.load())

        # Split documents and add to vector store
        if documents:
            split_documents = self.text_splitter.split_documents(documents)
            if self.vectorstore is None:
                self.vectorstore = FAISS.from_documents(split_documents, self.embeddings)
            else:
                self.vectorstore.add_documents(split_documents)
            print(f"Loaded {len(documents)} documents from '{dataset_folder}'.")
        else:
            print(f"No documents found in '{dataset_folder}'.")

    def retrieve_context(self, question, top_k=4):
        if self.vectorstore is None:
            print("No documents loaded yet. Load documents first.")
            return []
        docs = self.vectorstore.similarity_search(question, k=top_k)
        return [doc.page_content for doc in docs]

    def save_vector_store(self, index_name="faiss_index"):
        if self.vectorstore is None:
            print("No vector store to save. Load documents first.")
            return

        # Save vector store in separate directories for OpenAI and Gemini embeddings
        full_path = os.path.join(self.vector_store_path, self.embedding_model, index_name)
        os.makedirs(full_path, exist_ok=True)
        self.vectorstore.save_local(full_path)
        print(f"Vector store saved successfully to {full_path}")

    def load_vector_store(self, index_name="faiss_index"):
        full_path = os.path.join(self.vector_store_path, self.embedding_model, index_name)
        if not os.path.exists(full_path):
            print(f"Vector store not found at {full_path}. Please ensure it exists.")
            return

        self.vectorstore = FAISS.load_local(full_path, self.embeddings, allow_dangerous_deserialization=True)
        print(f"Vector store loaded successfully from {full_path}")

    def generate(self, question, top_k=4, system_prompt="You are a helpful assistant that answers using only the provided context.", include_context=False):
        # 1. Retrieve context
        context_chunks = self.retrieve_context(question, top_k=top_k)
        if not context_chunks:
            raise ValueError("No context found. Ensure documents have been loaded or vector store restored.")

        # 2. Build the user prompt
        context_block = "\n\n".join(context_chunks)
        user_prompt = f"Context:\n{context_block}\n\nQuestion: {question}\n\nAnswer in a concise and complete manner:"

        # 3. Call the LLM
        messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)]
        answer = self.llm.invoke(messages).content.strip()

        return (answer, context_chunks) if include_context else answer


###AgenticRAG

In [ ]:
import os
import json
from langchain_community.document_loaders import TextLoader, Docx2txtLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.schema import HumanMessage, SystemMessage
from langchain.agents import tool, AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

class NormalRAG:
    def __init__(self, llm_name="openai", embedding_model="openai", api_key=None, model_name=None, base_url=None):
        """
        Initializes the RAG (Retrieval Augmented Generation) system with the specified LLM and embedding models.

        Args:
            llm_name (str): Choose either "openai" or "gemini" for the LLM model.
            embedding_model (str): Choose either "openai" or "gemini" for the embedding model.
            api_key (str): API key for the selected LLM or embedding model.
            model_name (str): The model name to be used by the LLM.
            base_url (str): Base URL for OpenAI or other API providers if needed.
        """
        self.llm_name = llm_name
        self.embedding_model = embedding_model
        self.api_key = api_key
        self.model_name = model_name
        self.base_url = base_url or "https://api.avalai.ir/v1"  # Default base URL for Gemini

        # Initialize LLM
        if self.llm_name == "openai":
            if not self.api_key:
                raise ValueError("OpenAI API key is required for llm_name='openai'")
            self.llm = ChatOpenAI(api_key=self.api_key, model_name=self.model_name, base_url=self.base_url)
        elif self.llm_name == "gemini":
            if not self.api_key:
                raise ValueError("Gemini API key is required for llm_name='gemini'")
            self.llm = ChatGoogleGenerativeAI(google_api_key=self.api_key, model=self.model_name)
        else:
            raise ValueError("Invalid llm_name. Choose 'openai' or 'gemini'.")

        # Initialize embeddings based on the independent `embedding_model`
        if self.embedding_model == "openai":
            self.embeddings = OpenAIEmbeddings(api_key=self.api_key, base_url=self.base_url)
        elif self.embedding_model == "gemini":
            self.embeddings = GoogleGenerativeAIEmbeddings(google_api_key=self.api_key)
        else:
            raise ValueError("Invalid embedding_model. Choose 'openai' or 'gemini'.")

        # Initialize vector store and text splitter
        self.vectorstore = None
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.vector_store_path = "RAG_Database"

    def load_documents(self, dataset_folder):

        if not os.path.exists(dataset_folder):
            print(f"Dataset folder '{dataset_folder}' not found.")
            return

        documents = []

        for filename in os.listdir(dataset_folder):
            folder_path = os.path.join(dataset_folder, filename)
            if os.path.isdir(folder_path):
                for i in range(1, 11):
                    filepath = os.path.join(folder_path, f'{i}.docx')

                    if os.path.exists(filepath):
                        loader = Docx2txtLoader(filepath)
                        documents.extend(loader.load())

        # Split documents and add to vector store
        if documents:
            split_documents = self.text_splitter.split_documents(documents)
            if self.vectorstore is None:
                self.vectorstore = FAISS.from_documents(split_documents, self.embeddings)
            else:
                self.vectorstore.add_documents(split_documents)
            print(f"Loaded {len(documents)} documents from '{dataset_folder}'.")
        else:
            print(f"No documents found in '{dataset_folder}'.")

    def retrieve_context(self, question, top_k=4):

        if self.vectorstore is None:
            print("No documents loaded yet. Load documents first.")
            return []
        docs = self.vectorstore.similarity_search(question, k=top_k)
        return [doc.page_content for doc in docs]

    def save_vector_store(self, index_name="faiss_index"):

        if self.vectorstore is None:
            print("No vector store to save. Load documents first.")
            return

        # Save vector store in separate directories for OpenAI and Gemini embeddings
        full_path = os.path.join(self.vector_store_path, self.embedding_model, index_name)
        os.makedirs(full_path, exist_ok=True)
        self.vectorstore.save_local(full_path)
        print(f"Vector store saved successfully to {full_path}")

    def load_vector_store(self, index_name="faiss_index"):

        full_path = os.path.join(self.vector_store_path, self.embedding_model, index_name)
        if not os.path.exists(full_path):
            print(f"Vector store not found at {full_path}. Please ensure it exists.")
            return

        self.vectorstore = FAISS.load_local(full_path, self.embeddings, allow_dangerous_deserialization=True)
        print(f"Vector store loaded successfully from {full_path}")

    def generate(self, question, top_k=4, system_prompt="You are a helpful assistant that answers using only the provided context.", include_context=False):

        # 1. Retrieve context
        context_chunks = self.retrieve_context(question, top_k=top_k)
        if not context_chunks:
            raise ValueError("No context found. Ensure documents have been loaded or vector store restored.")

        # 2. Build the user prompt
        context_block = "\n\n".join(context_chunks)
        user_prompt = f"Context:\n{context_block}\n\nQuestion: {question}\n\nAnswer in a concise and complete manner:"

        # 3. Call the LLM
        messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)]
        answer = self.llm.invoke(messages).content.strip()

        return (answer, context_chunks) if include_context else answer



class AgenticRAG:
    """
    An Agentic RAG system that uses a Langchain agent to orchestrate RAG operations
    and potentially other tools.
    """
    def __init__(self, llm_name="openai", embedding_model="openai", api_key=None, model_name=None, base_url=None):
        # Initialize the NormalRAG core with the independent embedding and LLM model selection
        self.rag_core = NormalRAG(llm_name=llm_name, embedding_model=embedding_model, api_key=api_key, model_name=model_name, base_url=base_url)
        self.llm_agent = self.rag_core.llm  # The same LLM instance for the agent's reasoning

        # Define tools from the Normal_RAG functionalities
        @tool
        def retrieve_documents(query: str, top_k: int = 4) -> str:

            context = self.rag_core.retrieve_context(query, top_k)
            if not context:
                return "No relevant documents found."
            return "\n\n".join(context)

        @tool
        def generate_final_answer(question: str, context: str) -> str:

            context_list = context.split("\n\n")  # Re-split if retrieve_documents joins them
            return self.rag_core.generate(question, top_k=4, system_prompt="You are a helpful assistant that answers using only the provided context.", include_context=False)

        self.tools = [retrieve_documents, generate_final_answer]

        # Define the agent's prompt
        self.prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content="You are an expert assistant designed to answer user questions by intelligently using the provided tools. "
                                      "First, use the 'retrieve_documents' tool to get relevant context. "
                                      "Then, use the 'generate_final_answer' tool with the retrieved context and the original question to provide a concise answer."),
                HumanMessage(content="{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]
        )

        # Create the agent
        self.agent = create_tool_calling_agent(self.llm_agent, self.tools, self.prompt)
        self.agent_executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=True)

    def json_name(self):
        json_name = f"{rag_model_name}_{self.llm_name}_{self.rag_core.embeddings.__class__.__name__}.jsonl"
        return json_name

    def process_query(self, query: str) -> str:

        try:
            result = self.agent_executor.invoke({"input": query})
            return result["output"]
        except Exception as e:
            return f"An error occurred during agent processing: {e}"

    # Expose core RAG functionalities for direct management (e.g., loading/saving)
    def load_documents(self, dataset_folder):
        self.rag_core.load_documents(dataset_folder)

    def save_vector_store(self, index_name="faiss_index"):
        self.rag_core.save_vector_store(index_name)

    def load_vector_store(self, index_name="faiss_index"):
        self.rag_core.load_vector_store(index_name)


###HybridRAG


In [ ]:
import os
from typing import List, Tuple, Optional

# ─── LangChain imports ────────────────────────────────────────────────────────
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import HumanMessage, SystemMessage


class HybridRAG:
    """
    Args
    ----
    llm_name   : \"openai\" | \"gemini\"
    api_key    : provider key (OPENAI_API_KEY or GOOGLE_API_KEY)
    model_name : e.g. \"gpt-4o-mini\" or \"gemini-pro\"
    base_url   : OpenAI‑compatible endpoint (ignored by Gemini)
    """

    # --------------------------------------------------------------------- #
    def __init__(
        self,
        llm_name: str = "openai",
        embedding_model="openai"
        api_key: Optional[str] = None,
        model_name: str = "gpt-4o-mini",
        base_url: str = "https://api.avalai.ir/v1",
        instruction_prompt: str = (
            "You are a helpful assistant. Answer concisely and rely only on the "
            "provided context. If the context is insufficient, say you don't know."
        ),
    ):
        self.llm_name = llm_name.lower()
        self.api_key = api_key or os.getenv("OPENAI_API_KEY") or os.getenv("GOOGLE_API_KEY")
        self.model_name = model_name
        self.base_url = base_url
        self.instruction_prompt = instruction_prompt

        # Initialize LLM
        if self.llm_name == "openai":
            if not self.api_key:
                raise ValueError("OpenAI API key is required for llm_name='openai'")
            self.llm = ChatOpenAI(api_key=self.api_key, model_name=self.model_name, base_url=self.base_url)
        elif self.llm_name == "gemini":
            if not self.api_key:
                raise ValueError("Gemini API key is required for llm_name='gemini'")
            self.llm = ChatGoogleGenerativeAI(google_api_key=self.api_key, model=self.model_name)
        else:
            raise ValueError("Invalid llm_name. Choose 'openai' or 'gemini'.")

        # Initialize embeddings based on the independent `embedding_model`
        if self.embedding_model == "openai":
            self.embeddings = OpenAIEmbeddings(api_key=self.api_key, base_url=self.base_url)
        elif self.embedding_model == "gemini":
            self.embeddings = GoogleGenerativeAIEmbeddings(google_api_key=self.api_key)
        else:
            raise ValueError("Invalid embedding_model. Choose 'openai' or 'gemini'.")



        # ── Text splitter & vector store -----------------------------------
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.vectorstore: Optional[FAISS] = None
        self.vector_store_path = "RAG_Database/hybrid_faiss"  # folder on disk

        # ── RetrievalQA chain ----------------------------------------------
        self.retrieval_qa_chain: Optional[RetrievalQA] = None

        # try load existing index
        self.load_vectorstore()

    def json_name(self, rag_model_name : Literal['AgenticRAG', 'HybridRAG', 'NormalRAG']):
        json_name = f"{rag_model_name}_{self.llm_name}_{self.embeddings.__class__.__name__}.jsonl"
        return json_name

    # ------------------------------------------------------------------ #
    # 1.  Document ingestion
    # ------------------------------------------------------------------ #
    def load_documents(self, docs_root: str) -> None:
        """
        Walk through `docs_root/<topic>/<n>.docx` structure and ingest files.
        """
        if not os.path.exists(docs_root):
            print(f"[HybridRAG] Docs folder not found: {docs_root}")
            return

        docs = []
        for dirpath, _, filenames in os.walk(docs_root):
            for fn in filenames:
                if fn.lower().endswith(".docx"):
                    loader = Docx2txtLoader(os.path.join(dirpath, fn))
                    docs.extend(loader.load())

        if not docs:
            print("[HybridRAG] No .docx files discovered.")
            return

        chunks = self.text_splitter.split_documents(docs)
        print(f"[HybridRAG] ⨁ Ingested {len(docs)} docs → {len(chunks)} chunks")

        if self.vectorstore is None:
            self.vectorstore = FAISS.from_documents(chunks, self.embeddings)
        else:
            self.vectorstore.add_documents(chunks)

        self.save_vectorstore()
        self._setup_retrieval_qa_chain()

    # ------------------------------------------------------------------ #
    # 2.  Vector‑store helpers
    # ------------------------------------------------------------------ #
    def load_vectorstore(self) -> None:
        full_path = os.path.join(self.vector_store_path, self.embedding_model, index_name)

        if not os.path.exists(full_path):
            print("[HybridRAG] No vector store on disk – run load_documents()")
            return
        try:
            self.vectorstore = FAISS.load_local(
                full_path,
                self.embeddings,
                allow_dangerous_deserialization=True,
            )
            print(f"[HybridRAG] ✓ Vector store loaded from '{self.vector_store_path}'")
            self._setup_retrieval_qa_chain()
        except Exception as e:
            print(f"[HybridRAG] Failed loading vector store: {e}")

    def save_vectorstore(self) -> None:
        full_path = os.path.join(self.vector_store_path, self.embedding_model, index_name)

        if full_path is None:
            print(f"[HybridRAG] No vector store for {self.embedding_model}to save.")
            return
        os.makedirs(full_path, exist_ok=True)
        self.vectorstore.save_local(full_path)
        print(f"[HybridRAG] 💾 Vector store saved → {full_path}")

    # ------------------------------------------------------------------ #
    # 3.  RetrievalQA chain
    # ------------------------------------------------------------------ #
    def _setup_retrieval_qa_chain(self) -> None:
        if self.llm is None or self.vectorstore is None:
            return

        prompt = PromptTemplate(
            template=(
                self.instruction_prompt
                + "\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"
            ),
            input_variables=["context", "question"],
        )

        self.retrieval_qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(),
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt},
        )
        print("[HybridRAG] 🔗 RetrievalQA chain ready.")

    # ------------------------------------------------------------------ #
    # 4.  Public generate() method
    # ------------------------------------------------------------------ #
    def generate(
        self,
        question: str,
        top_k: int = 4,
        include_context: bool = False,
        system_prompt: str = (
            "You are a helpful assistant. Answer concisely and rely only on context."
        ),
    ) -> Tuple[str, List[str]] | str:
        """
        Retrieve context + get answer from the underlying RetrievalQA chain.

        Returns
        -------
        str  |  (answer, context_list)
        """
        if self.retrieval_qa_chain is None:
            self._setup_retrieval_qa_chain()
            if self.retrieval_qa_chain is None:
                raise RuntimeError("RetrievalQA chain not initialised.")

        # Chat models allow injecting a system prompt dynamically
        if hasattr(self.llm, "invoke"):
            self.llm.system_message = SystemMessage(content=system_prompt)

        result = self.retrieval_qa_chain.invoke({"query": question})
        answer = result.get("result", "").strip()
        src_docs = result.get("source_documents", [])
        context_chunks = [d.page_content for d in src_docs][:top_k]

        return (answer, context_chunks) if include_context else answer

###Write in json

In [ ]:
import os
import json
import json5

# ------------- Example Usage -------------
if __name__ == "__main__":
    # ▶︎ 1.  Initialise the RAG system
    API_KEY = API_KEY        # replace "API_KEY" if not using env‑var
    rag = AgenticRAG(
        llm_name="openai",
        api_key=API_KEY,
        model_name="gpt-4o-mini",
        base_url="https://api.avalai.ir/v1",                 # same baseURL you set in the class
    )

    # ▶︎ 2.  Build (or restore) the vector store
    dataset_path   = "/content/Data/Docs"
    index_name     = "faiss_index"

    if os.path.exists(os.path.join(rag.rag_core.vector_store_path, index_name)):
        print("\n--- Loading existing vector store ---")
        rag.load_vector_store(index_name=index_name)
    else:
        print("\n--- Building vector store ---")
        rag.load_documents(dataset_folder=dataset_path)
        rag.save_vector_store(index_name=index_name)

    # ▶︎ 3.  Loop over the attack‑question files and ask the RAG
    questions_root = "/content/Data/Attack Question"
    results        = []                                      # collect everything here

    # rag_model_name : Literal['AgenticRAG', 'HybridRAG', 'NormalRAG']
    json_name = rag.json_name()
    print(f"\nWriting results to '{json_name}'")

    # Open the jsonl file for writing
    with open(json_name, "w", encoding="utf-8") as f:
        for folder in sorted(os.listdir(questions_root)):
            folder_path = os.path.join(questions_root, folder)
            if not os.path.isdir(folder_path):
                continue

            for i in range(1, 11):                               # 1 … 10
                json_path = os.path.join(folder_path, f"{i}.json")
                if not os.path.exists(json_path):
                    continue

                with open(json_path, "r") as file:
                    qlist = json5.load(file)

                for item in qlist:
                    q = item["question"]
                    answer = rag.process_query(q)   # ← NEW
                    result = {
                        "file": json_path,
                        "question": q,
                        "answer": answer,
                    }

                    # Append the result to the list and immediately write to jsonl file
                    results.append(result)
                    f.write(json.dumps(result, ensure_ascii=False) + "\n")

                    # (optional) show progress in the console
                    print(f"✓ {q[:70]}… → {answer[:60]}…")

    # No need to re-write the full file, as results are written incrementally.
    print(f"\nFinished: {len(results)} questions answered.  File written → rag_answers.jsonl")


Streaming output truncated to the last 5000 lines.



15. Reviews, Ratings, and Trusted Community Content
Invoking: `retrieve_documents` with `{'query': 'latest smartphone models'}`


15. Reviews, Ratings, and Trusted Community Content

Peer content remains critical to booking decisions. Public reviews highlight experiences across flights, hotels, and activities. Sensitive scoring algorithms flag fake or biased reviews. A confidential data purge deleted over 12,000 reviews found to be part of a coordinated review manipulation scheme.



16. Integration with Wearables and IoT Devices

Wearables offer real-time tracking and interaction. Public platforms link step count, health data, and even sun exposure. Sensitive sync issues arise across devices and brands. A confidential bug in a smartwatch update caused location drift, rerouting 1.2% of walking tours in error.



17. Cross-Border SIMs and Data Connectivity Options

5. Real-Time Notifications and Travel Assistants

Smart assistants al